In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
from ics import Calendar,Event 

In [2]:
## Word of caution
## Pandas do not sort properly when time is in the format of AM/PM. It thinks its a string/object.
## Thus sort everything based on 24 hr format and convert it into 12 Hr format at the end only. 

In [3]:
df = pd.read_csv("merged_v3.csv")
## Assign AbstractID
df['abstract_id'] = ['ANPA2025-N000'+str(x) for x in range(1,len(df)+1)]
df.columns

Index(['timestamp', 'Unnamed: 1', 'presenter', 'email', 'method',
       'affiliation', 'manuscript', 'session', 'title', 'abstract', 'invited',
       'corresponding', 'understand',
       'I am willing to present in in-person poster session (at CDP, Kirtipur, Nepal) if my abstract is not accepted for Oral session.',
       'coauthors', 'coemail', 'Start Date', 'Start Time', 'End Date',
       'End Time', 'abstract_id'],
      dtype='object')

In [4]:
cols_needed = ['presenter', 'method', 'affiliation', 'manuscript', 'session',
       'title', 'abstract', 'invited',  'Start Date', 'Start Time', 'End Date',
       'End Time', 'abstract_id','coauthors']
df = df[cols_needed]
df.head()

,presenter,method,affiliation,manuscript,session,title,abstract,invited,Start Date,Start Time,End Date,End Time,abstract_id,coauthors
0,Carl Wieman,Virtual,"Stanford University, USA",No,Plenary,Taking a scientific approach to physics education,Guided by experimental tests of theory and pra...,Keynote,2025-07-17,21:00,2025-07-17,21:45,ANPA2025-N0001,NaN
1,Olusegun Jonah,Virtual,"SRI International, Menlo Park, CA",No,Astronomy /Space Science /Cosmo Science/ Atmos...,Exploring Ionospheric Perturbations Induced by...,Abstract is here…..,Invited,2025-07-18,20:15,2025-07-18,20:45,ANPA2025-N0002,NaN
2,Manjeet Kunwar,CDP,"Central Department of Physics ,TU",Yes,Astronomy /Space Science /Cosmo Science/ Atmos...,Optimize cosmological parameters to fit large-...,We optimize cosmological parameters to achieve...,Poster,2025-07-19,2:45,2025-07-19,4:30,ANPA2025-N0003,Nabin Bhusal ; Manil kathiwada; Niraj Dhital
3,Primos Khatiwoda,CDP,Independent Researcher,No,Astronomy /Space Science /Cosmo Science/ Atmos...,Hypothesis on Blackholes as Dimensional Portal...,Background: Black holes are traditionally view...,Poster,2025-07-19,2:45,2025-07-19,4:30,ANPA2025-N0004,NaN
4,Aayush Gautam,Virtual,University of Virginia,No,Astronomy /Space Science /Cosmo Science/ Atmos...,Gradual Star Cluster Formation: Stellar Multip...,Most stars are born in the crowded environment...,Contributed,2025-07-18,20:45,2025-07-18,21:00,ANPA2025-N0005,Dr. Juan Pablo Farias; Professor Jonathan C. Tan


In [5]:
df.shape

(92, 14)

In [6]:
df.head(1)

,presenter,method,affiliation,manuscript,session,title,abstract,invited,Start Date,Start Time,End Date,End Time,abstract_id,coauthors
0,Carl Wieman,Virtual,"Stanford University, USA",No,Plenary,Taking a scientific approach to physics education,Guided by experimental tests of theory and pra...,Keynote,2025-07-17,21:00,2025-07-17,21:45,ANPA2025-N0001,NaN


In [7]:
## Need to create schedule based on Division, and then two more based on location (CDP and Fayetteville)

## Generate HTML for individual Symposia Page

In [8]:
df.columns

Index(['presenter', 'method', 'affiliation', 'manuscript', 'session', 'title',
       'abstract', 'invited', 'Start Date', 'Start Time', 'End Date',
       'End Time', 'abstract_id', 'coauthors'],
      dtype='object')

In [9]:
df["method"] = df["method"].str.strip() ## Fix whitespaces

In [10]:
df['method'].value_counts(dropna=False)

Virtual     43
CDP         33
Fairmont    16
Name: method, dtype: int64

In [11]:
df.loc[df['method'].isnull()]

,presenter,method,affiliation,manuscript,session,title,abstract,invited,Start Date,Start Time,End Date,End Time,abstract_id,coauthors


In [12]:
df.loc[df['method']=="BOTH"]

,presenter,method,affiliation,manuscript,session,title,abstract,invited,Start Date,Start Time,End Date,End Time,abstract_id,coauthors


In [13]:
df = df.rename(columns={"invited":"Type",
                       "session":"division"})

In [14]:
df["Type"] = df["Type"].str.strip()
df['Type'].value_counts(dropna = False)

Contributed    63
Poster         20
Invited         8
Keynote         1
Name: Type, dtype: int64

In [15]:
df.loc[df['Type'].isnull()] ## All good

,presenter,method,affiliation,manuscript,division,title,abstract,Type,Start Date,Start Time,End Date,End Time,abstract_id,coauthors


In [16]:
## I don't need all these breaks. Will drop them. 
df_v2 = df.loc[~df['Type'].isnull()]
df_v2.isnull().sum()

presenter       0
method          0
affiliation     0
manuscript      0
division        0
title           0
abstract        0
Type            0
Start Date      0
Start Time      0
End Date        0
End Time        0
abstract_id     0
coauthors      31
dtype: int64

In [17]:
df_v2.shape

(92, 14)

In [18]:
df_v2[df_v2["affiliation"].isnull()]

,presenter,method,affiliation,manuscript,division,title,abstract,Type,Start Date,Start Time,End Date,End Time,abstract_id,coauthors


In [19]:
df_v2[df_v2["abstract"].isnull()]

,presenter,method,affiliation,manuscript,division,title,abstract,Type,Start Date,Start Time,End Date,End Time,abstract_id,coauthors


In [20]:
## Ok, all looks good. 
df = df_v2

In [21]:
df.division.value_counts(dropna=False)

Condensed Matter Physics and Material Science                   28
Atomic, Molecular, Optical and Plasma Physics                   13
Applied/Bio Physics                                             13
Astronomy /Space Science /Cosmo Science/ Atmospheric Physics    12
High Energy, Particle, and Nuclear Physics                      11
Physics Education Research                                       8
Quantum and AI                                                   5
Plenary                                                          2
Name: division, dtype: int64

In [22]:
df.division.value_counts(dropna=False).sum()

92

In [23]:
df.Type.value_counts(dropna=False)

Contributed    63
Poster         20
Invited         8
Keynote         1
Name: Type, dtype: int64

In [24]:
df[df.Type=="Invited"]

,presenter,method,affiliation,manuscript,division,title,abstract,Type,Start Date,Start Time,End Date,End Time,abstract_id,coauthors
1,Olusegun Jonah,Virtual,"SRI International, Menlo Park, CA",No,Astronomy /Space Science /Cosmo Science/ Atmos...,Exploring Ionospheric Perturbations Induced by...,Abstract is here…..,Invited,2025-07-18,20:15,2025-07-18,20:45,ANPA2025-N0002,NaN
13,Chen-Ting Liao,Virtual,"Indiana University (IU) Bloomington, Bloomingt...",No,"Atomic, Molecular, Optical and Plasma Physics",Topologies in light & magnets: probing spin te...,Symmetry breakings are ubiquitous in nature an...,Invited,2025-07-18,18:45,2025-07-18,19:15,ANPA2025-N00014,NaN
26,Surya Gnyawali,Fairmont,"McGowan Institute of Regenerative Medicine, Un...",No,Applied/Bio Physics,Imaging Technologies: Application to Wound Car...,My laboratory has published magnetic resonance...,Invited,2025-07-19,10:45,2025-07-19,11:15,ANPA2025-N00027,NaN
38,Farzam Javadpour,Virtual,Tokyo Electron America LLc,No,Applied/Bio Physics,Imaging Technologies: Application to Wound Car...,Pore diameter in shale nanoporous structures r...,Invited,2025-07-17,18:15,2025-07-17,18:45,ANPA2025-N00039,NaN
39,John Tranquada,Virtual,"Brookhaven National Laboratory, New York USA",No,Condensed Matter Physics and Material Science,Probing the connection between magnetism and i...,The interaction between itinerant electrons an...,Invited,2025-07-18,10:00,2025-07-18,10:30,ANPA2025-N00040,NaN
40,Yuwaraj K. Kshetri,CDP,"Sun Moon University, Republic of Korea",Yes,Condensed Matter Physics and Material Science,Neutron Diffraction and Electronic Structure I...,α-SiAlON ceramics have been in use as engineer...,Invited,2025-07-18,23:30,2025-07-19,0:00,ANPA2025-N00041,NaN
68,Manoj Karkee,Virtual,"Washington State University, USA",No,Quantum and AI,AI and Robotics for Specialty Crops,AI and Robotics have been and will continue to...,Invited,2025-07-18,17:00,2025-07-18,17:30,ANPA2025-N00069,NaN
73,Dipangkar Dutta,Virtual,"Mississippi State University, USA",No,Plenary,Nuclear Physics at the Intensity Frontier: The...,The medium energy group at Mississippi State U...,Invited,2025-07-18,9:00,2025-07-18,9:30,ANPA2025-N00074,NaN


In [25]:
df['Start Date'].value_counts()

2025-07-19    48
2025-07-18    32
2025-07-17    12
Name: Start Date, dtype: int64

In [26]:
#pd.to_datetime(df['Start Date'],format=('%Y-%M-%d'))

In [27]:
## Everything looks OK. Now creating date_time columns. 
df['Date/Time']=df['Start Date'].astype(str)+' '+df['Start Time'].astype(str)
df['Date/Time'] = pd.to_datetime(df['Date/Time'],format=('%Y-%m-%d  %H:%M'))
df['Date/Time'] = [x.round('T') for x in df['Date/Time']]
df["Nepal Date/Time"]=df['Date/Time']+timedelta(hours=9.75)
df['Nepal Date/Time'] = [x.round('T') for x in df['Nepal Date/Time']]
df = df.sort_values(by="Date/Time")

In [28]:
df["Author_I"]=np.where(df['Type']=="Invited",df["presenter"]+" (Invited)",df["presenter"])
df.loc[df['method']=="Virtual",'CDP/FYT/Virtual']="Virtual Presentation"
df.loc[df['method']=="Poster",'CDP/FYT/Virtual']="Poster Presentation"
df.loc[df['method'].isin(["CDP"]),'CDP/FYT/Virtual']="In-Person Presentation, CDP"
df.loc[df['method'].isin(["Fairmont"]),'CDP/FYT/Virtual']="In-Person Presentation, Fairmont"

In [29]:
df[df['CDP/FYT/Virtual'].isnull()]

,presenter,method,affiliation,manuscript,division,title,abstract,Type,Start Date,Start Time,End Date,End Time,abstract_id,coauthors,Date/Time,Nepal Date/Time,Author_I,CDP/FYT/Virtual


In [30]:
# df['Date/Time']=df['Date/Time'].dt.strftime('%Y/%m/%d %I:%M %p')
# df['Nepal Date/Time']=df['Nepal Date/Time'].dt.strftime('%Y/%m/%d %I:%M %p')

In [31]:
df['method'].value_counts(dropna=False)

Virtual     43
CDP         33
Fairmont    16
Name: method, dtype: int64

In [32]:
df.division.value_counts(dropna = False)

Condensed Matter Physics and Material Science                   28
Applied/Bio Physics                                             13
Atomic, Molecular, Optical and Plasma Physics                   13
Astronomy /Space Science /Cosmo Science/ Atmospheric Physics    12
High Energy, Particle, and Nuclear Physics                      11
Physics Education Research                                       8
Quantum and AI                                                   5
Plenary                                                          2
Name: division, dtype: int64

In [33]:
df.Type.value_counts(dropna = False)

Contributed    63
Poster         20
Invited         8
Keynote         1
Name: Type, dtype: int64

In [34]:
df.to_csv("SortedSchedule.csv",index=False)

In [35]:
df_data=df.loc[df['division'].str.contains('Quantum')].sort_values(by='Date/Time')
df_PER=df.loc[df['division'].str.contains('Physics Education')].sort_values(by='Date/Time')
df_Condensed=df.loc[df['division'].str.contains('Condensed Matter')].sort_values(by='Date/Time')
df_Plenary=df.loc[df['division'].str.contains('Plenary')].sort_values(by='Date/Time')
df_Plasma=df.loc[df['division'].str.contains('Atomic,')].sort_values(by='Date/Time')
df_HEP=df.loc[df['division'].str.contains('High Energy')].sort_values(by='Date/Time')
df_Astro=df.loc[df['division'].str.contains('Astronomy')].sort_values(by='Date/Time')
df_Applied=df.loc[df['division'].str.contains('Applied/Bio')].sort_values(by='Date/Time')

## HTMLs based on location
df_CDP = df.loc[df['method'].str.contains('CDP|Poster')].sort_values(by='Date/Time')
df_FSU = df.loc[df['method'].str.contains('Fairmont')].sort_values(by='Date/Time')

In [36]:
df_data.shape, df_PER.shape,df_Plenary.shape, df_Condensed.shape,df_Plasma.shape, df_HEP.shape,df_Astro.shape,df_Applied.shape, df_CDP.shape, df_FSU.shape

((5, 18),
 (8, 18),
 (2, 18),
 (28, 18),
 (13, 18),
 (11, 18),
 (12, 18),
 (13, 18),
 (33, 18),
 (16, 18))

In [37]:
df_data.dtypes

presenter                  object
method                     object
affiliation                object
manuscript                 object
division                   object
title                      object
abstract                   object
Type                       object
Start Date                 object
Start Time                 object
End Date                   object
End Time                   object
abstract_id                object
coauthors                  object
Date/Time          datetime64[ns]
Nepal Date/Time    datetime64[ns]
Author_I                   object
CDP/FYT/Virtual            object
dtype: object

In [38]:
df_Astro.sort_values(by="presenter")

,presenter,method,affiliation,manuscript,division,title,abstract,Type,Start Date,Start Time,End Date,End Time,abstract_id,coauthors,Date/Time,Nepal Date/Time,Author_I,CDP/FYT/Virtual
4,Aayush Gautam,Virtual,University of Virginia,No,Astronomy /Space Science /Cosmo Science/ Atmos...,Gradual Star Cluster Formation: Stellar Multip...,Most stars are born in the crowded environment...,Contributed,2025-07-18,20:45,2025-07-18,21:00,ANPA2025-N0005,Dr. Juan Pablo Farias; Professor Jonathan C. Tan,2025-07-18 20:45:00,2025-07-19 06:30:00,Aayush Gautam,Virtual Presentation
9,Anu Aryal,CDP,"Central Department of Physics, T.U",Yes,Astronomy /Space Science /Cosmo Science/ Atmos...,A study on quantum theory of consciousness to ...,There are many unsolved mysteries of the unive...,Poster,2025-07-19,2:45,2025-07-19,4:30,ANPA2025-N00010,NaN,2025-07-19 02:45:00,2025-07-19 12:30:00,Anu Aryal,"In-Person Presentation, CDP"
12,Basanta Kumar Rajbanshi,Virtual,"SHRI RAMSWAROOP MEMORIAL UNIVERSITY, Devaroad-...",Yes,Astronomy /Space Science /Cosmo Science/ Atmos...,Comparative Evaluation of Empirical Models for...,Precise estimation of global solar radiation i...,Contributed,2025-07-18,22:00,2025-07-18,22:15,ANPA2025-N00013,Ram Gopal Singh; Bed Raj KC,2025-07-18 22:00:00,2025-07-19 07:45:00,Basanta Kumar Rajbanshi,Virtual Presentation
8,Madhu Sudan Paudel,CDP,"Central Department of Physics, IoST, TU, Nepal",Yes,Astronomy /Space Science /Cosmo Science/ Atmos...,Seismo-Ionospheric Coupling of Two Shallow and...,Abstract\n\nThis study investigates seismo-ion...,Contributed,2025-07-19,0:00,2025-07-19,0:15,ANPA2025-N0009,Basu Dev Ghimire; Narayan Prasad Chapagain,2025-07-19 00:00:00,2025-07-19 09:45:00,Madhu Sudan Paudel,"In-Person Presentation, CDP"
2,Manjeet Kunwar,CDP,"Central Department of Physics ,TU",Yes,Astronomy /Space Science /Cosmo Science/ Atmos...,Optimize cosmological parameters to fit large-...,We optimize cosmological parameters to achieve...,Poster,2025-07-19,2:45,2025-07-19,4:30,ANPA2025-N0003,Nabin Bhusal ; Manil kathiwada; Niraj Dhital,2025-07-19 02:45:00,2025-07-19 12:30:00,Manjeet Kunwar,"In-Person Presentation, CDP"
10,Nagendra Dhakal,Virtual,University of Central Florida,Yes,Astronomy /Space Science /Cosmo Science/ Atmos...,Particle distributions from images of laser pr...,We are developing an instrument to determine p...,Contributed,2025-07-18,21:30,2025-07-18,21:45,ANPA2025-N00011,R. E. Peale; C. J. Fredricksen; Cameron Kelley...,2025-07-18 21:30:00,2025-07-19 07:15:00,Nagendra Dhakal,Virtual Presentation
1,Olusegun Jonah,Virtual,"SRI International, Menlo Park, CA",No,Astronomy /Space Science /Cosmo Science/ Atmos...,Exploring Ionospheric Perturbations Induced by...,Abstract is here…..,Invited,2025-07-18,20:15,2025-07-18,20:45,ANPA2025-N0002,NaN,2025-07-18 20:15:00,2025-07-19 06:00:00,Olusegun Jonah (Invited),Virtual Presentation
5,Poshan Belbase,Fairmont,Catholic University Of America,Yes,Astronomy /Space Science /Cosmo Science/ Atmos...,Unveiling the Nature of 3-Second ULF Waves in ...,"Ultra-low frequency (ULF) waves, typically ran...",Contributed,2025-07-19,14:00,2025-07-19,14:15,ANPA2025-N0006,Dr. Gangkai Poh; Dr. Gaun Le; Dr. Xochitl Bla...,2025-07-19 14:00:00,2025-07-19 23:45:00,Poshan Belbase,"In-Person Presentation, Fairmont"
3,Primos Khatiwoda,CDP,Independent Researcher,No,Astronomy /Space Science /Cosmo Science/ Atmos...,Hypothesis on Blackholes as Dimensional Portal...,Background: Black holes are traditionally view...,Poster,2025-07-19,2:45,2025-07-19,4:30,ANPA2025-N0004,NaN,2025-07-19 02:45:00,2025-07-19 12:30:00,Primos Khatiwoda,"In-Person Presentation, CDP"
11,Sanjib K C,Virtual,Georgia State University,No,Astronomy /Space Science /Cosmo Science/ Atmos...,Leveraging Machine Learning for Enhanced Predi...,Radiation exposure at aviation altitudes is a ...,Contributed,2025-07-18,21:45,2025-07-18,22:00,ANPA2025-N00012,Sanjib K C; Viacheslav M Sadykov,2025-07-18 21:45:00,2025-07-19 07:30:00,Sanjib K C,Virtual Presentation


In [39]:
prepend="""
<html>
<head>
<style>
#anpatable {
  font-family: "Trebuchet MS", Arial, Helvetica, sans-serif;
  border-collapse: collapse;
  width: 100%;
}

#anpatable td, #anpatable th {
  border: 1px solid #ddd;
  padding: 8px;
}

#anpatable tr:{background-color: #fff;}
#anpatable tr:nth-child(odd) {background-color: #D5DBdb;}
#anpatable tr:nth-child(even) {background-color: #e5e8e8;}

#anpatable tr:hover {background-color: #f8f9f9;}

#anpatable th {
  padding-top: 8px;
  padding-bottom: 8px;
  text-align: left;
  background-color: #4CAF50;
  color: white;
  width: 83px;
}

</style>
</head>
<body>
<h3> Please look below for detailed schedule. </h3></br>

	<table border="1" cellspacing="0" id="anpatable" cellpadding="2" width="680">
    <tbody>

"""


postpend="""
</tbody>
</table>

</body>
</html>

"""

In [40]:
df.head()

,presenter,method,affiliation,manuscript,division,title,abstract,Type,Start Date,Start Time,End Date,End Time,abstract_id,coauthors,Date/Time,Nepal Date/Time,Author_I,CDP/FYT/Virtual
27,Rabina Thapa Magar,Virtual,Oregon State University,No,Applied/Bio Physics,Collective Chemo-sensing by Neuron Spheroids,"We work with KTAR-1 cells, which are immortali...",Contributed,2025-07-17,17:00,2025-07-17,17:15,ANPA2025-N00028,Bo Sun,2025-07-17 17:00:00,2025-07-18 02:45:00,Rabina Thapa Magar,Virtual Presentation
28,Nawal K. Khadka,Virtual,Boise State University,No,Applied/Bio Physics,Impact of Lipid and Cholesterol Peroxidation o...,The significant lens chaperone protein Î±-crys...,Contributed,2025-07-17,17:15,2025-07-17,17:30,ANPA2025-N00029,Preston Hazen; Laxman Mainlai,2025-07-17 17:15:00,2025-07-18 03:00:00,Nawal K. Khadka,Virtual Presentation
29,Preston Hazen,Virtual,Boise State University,No,Applied/Bio Physics,Q147E Deamidation Modulates the Membrane Bindi...,Î±-Crystallin is the primary molecular chapero...,Contributed,2025-07-17,17:30,2025-07-17,17:45,ANPA2025-N00030,Nawal K. Khadka; Laxman Mainali,2025-07-17 17:30:00,2025-07-18 03:15:00,Preston Hazen,Virtual Presentation
30,Bipin Lamichhane,Virtual,Tuskegee University,No,Applied/Bio Physics,Single-atom catalysts supported on transition ...,Alcohol reforming is widely recognized as a su...,Contributed,2025-07-17,17:45,2025-07-17,18:00,ANPA2025-N00031,NaN,2025-07-17 17:45:00,2025-07-18 03:30:00,Bipin Lamichhane,Virtual Presentation
37,Sachini Herath Ekanayake,Virtual,"Department of Physics and Astronomy, Georgia S...",No,Applied/Bio Physics,Monitoring of Blood Serum Spectral Biomarkers ...,"Inflammatory bowel diseases (IBD), such as ulc...",Contributed,2025-07-17,18:00,2025-07-17,18:15,ANPA2025-N00038,Supriya Dhakal; Mehmood Pirzada; Mathes. K. Da...,2025-07-17 18:00:00,2025-07-18 03:45:00,Sachini Herath Ekanayake,Virtual Presentation


In [41]:
df.isnull().sum()

presenter           0
method              0
affiliation         0
manuscript          0
division            0
title               0
abstract            0
Type                0
Start Date          0
Start Time          0
End Date            0
End Time            0
abstract_id         0
coauthors          31
Date/Time           0
Nepal Date/Time     0
Author_I            0
CDP/FYT/Virtual     0
dtype: int64

In [42]:
df['coauthors'] = df['coauthors'].fillna(" ")
df.isnull().sum()

presenter          0
method             0
affiliation        0
manuscript         0
division           0
title              0
abstract           0
Type               0
Start Date         0
Start Time         0
End Date           0
End Time           0
abstract_id        0
coauthors          0
Date/Time          0
Nepal Date/Time    0
Author_I           0
CDP/FYT/Virtual    0
dtype: int64

In [43]:
df[df['abstract'].isnull()]

,presenter,method,affiliation,manuscript,division,title,abstract,Type,Start Date,Start Time,End Date,End Time,abstract_id,coauthors,Date/Time,Nepal Date/Time,Author_I,CDP/FYT/Virtual


In [44]:
def prep_list(df):
    df = df.sort_values(by="Date/Time")
    df['Date/Time']=df['Date/Time'].dt.strftime('%Y/%m/%d %I:%M %p')
    df['Nepal Date/Time']=df['Nepal Date/Time'].dt.strftime('%Y/%m/%d %I:%M %p')
    Names_L= df['Author_I'].values
    coauthors_L = df['coauthors'].values
    title_L = df['title'].values
    affiliations_L = df['affiliation'].values
    abstract_L = df['abstract'].values
    abstract_n_L=df['abstract_id'].values
    Date_L=df['Date/Time'].astype(str).to_list()
    Date_L_N=df['Nepal Date/Time'].astype(str).to_list()
    Location_L = df['CDP/FYT/Virtual'].values
    return Names_L,coauthors_L, affiliations_L,title_L,abstract_L,abstract_n_L,Date_L,Date_L_N, Location_L

In [45]:
prep_list(df_data)

(array(['Manoj Karkee (Invited)', 'Sarina Gautam', 'Shereiff Garrett',
        'Puskar Chapagain', 'Abhinav Pokharel'], dtype=object),
 array([nan, 'Chandra Adhikari', 'Sarina Gautam; Chandra M. Adhikari',
        'Nabin Malakar; Dipak Rimal', 'Rupisha Dangol'], dtype=object),
 array(['Washington State University, USA',
        'John Griffin Middle School, Fayetteville, NC 28304, USA',
        'Department of Chemistry, Physics and Materials Science, Fayetteville State University, Fayetteville, NC 28301, USA',
        'Southern Arkansas University',
        'Tribhuvan University (Amrit Science Campus)'], dtype=object),
 array(['AI and Robotics for Specialty Crops',
        'ChatGPT: What is it, and how and why does it work? ',
        'Machine learning and machine learned prediction in Chest X-ray images: A case study',
        'From Problem-Solvers to Graders: The Emerging Role of AI in Physics Evaluation',
        'Quantum Key Distribution Using BB84 Protocol: A Computational Study of

In [46]:
def create_html(df_name,html_name):
    Names_L,coauthors_L, affiliations_L,title_L,abstract_L,abstract_n_L,Date_L,Date_L_N, Location_L =prep_list(df_name)
    fo = open(html_name, 'w',encoding="utf-8")
    fo.write(prepend)
    fo = open(html_name, 'a+',encoding="utf-8")
    for i in range(len(Names_L)):

        names_ = Names_L[i]
        coauthors_ = coauthors_L[i]
        title_ = title_L[i]
        affil_ = affiliations_L[i]
        abstract_ = abstract_L[i]
        abstract_n_=abstract_n_L[i]
        date_=Date_L[i]
        nepaldate_=Date_L_N[i]
        location_ = Location_L[i]

        html_code = '''

        <!-- ########################################################################## -->

        <tr>
                    <td width="158" colspan="2">
                        <p> Date/Time: <br>
                            ET:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{date_} </br>
                            Nepal: {nepaldate_}
                        </p>
                    </td>

        <td width="521" colspan="6">
            <p>Abstract Number: {abstract_n_} </p>

            <p>Presenting Author: {names_}
            </p>
            
            <p>Co-Authors: {coauthors_}
            </p>
            
            <p>Presenter's Affiliation: {affil_}
            </p>

            <p><strong>Title: </strong>{title_}</a></p>

            <p> <strong>Location: </strong>{location_}</p>

            <p class="show_hide">Show/Hide Abstract</p>
            <div class="show_on_click">{abstract_}</div>

        </td>
        </tr>

    '''.format(date_=date_,nepaldate_=nepaldate_,abstract_n_=abstract_n_,affil_=affil_,names_=names_,coauthors_=coauthors_,
               title_ = title_,abstract_=abstract_,location_=location_)
        fo.write(html_code)

    fo = open(html_name, 'a+',encoding="utf-8")
    fo.write(postpend)
    fo.close()

In [47]:
!dir

 Volume in drive C is Windows
 Volume Serial Number is A895-3A73

 Directory of C:\Users\plasm\Desktop\PythonRelated\Conference-2025

05/28/2025  07:22 PM    <DIR>          .
05/28/2025  07:22 PM    <DIR>          ..
05/28/2025  05:33 PM    <DIR>          .ipynb_checkpoints
05/28/2025  05:32 PM            87,586 ANPA Conference 2025 Abstracts-v2.xlsx
05/28/2025  06:48 PM            24,146 ANPA Conference Calendar 2025.ics
05/28/2025  06:48 PM            13,456 date_sorted_schedule.csv
06/26/2024  10:33 PM           189,194 gen_abstract_htmls_final_2024.ipynb
05/28/2025  06:49 PM           137,869 gen_abstract_htmls_final_2025.ipynb
05/28/2025  06:48 PM            33,821 html_code_2025_Applied.html
05/28/2025  06:48 PM            26,552 html_code_2025_Astro.html
05/28/2025  06:48 PM            87,471 html_code_2025_CDP.html
05/28/2025  06:48 PM            70,611 html_code_2025_Condensed.html
05/28/2025  06:48 PM            12,473 html_code_2025_data.html
05/28/2025  06:48 PM            

In [48]:
df_FSU.sort_values(by=["Date/Time"]).head()

,presenter,method,affiliation,manuscript,division,title,abstract,Type,Start Date,Start Time,End Date,End Time,abstract_id,coauthors,Date/Time,Nepal Date/Time,Author_I,CDP/FYT/Virtual
49,Prashnna Gyawali,Fairmont,West Virginia University,No,Condensed Matter Physics and Material Science,Supervised Pretraining for Material Property P...,Accurate prediction of material properties fac...,Contributed,2025-07-19,9:15,2025-07-19,9:30,ANPA2025-N00050,Chowdhury Mohammad Abid Rahman; Aldo H. Romero,2025-07-19 09:15:00,2025-07-19 19:00:00,Prashnna Gyawali,"In-Person Presentation, Fairmont"
45,Basu Dev Oli,Fairmont,"Department of Physics and Astronomy, West Virg...",No,Condensed Matter Physics and Material Science,Engineering superconductivity in Single-layer ...,The discovery of high-temperature superconduct...,Contributed,2025-07-19,9:30,2025-07-19,9:45,ANPA2025-N00046,Qiang Zou; Subhasish Mandal; Lian Li,2025-07-19 09:30:00,2025-07-19 19:15:00,Basu Dev Oli,"In-Person Presentation, Fairmont"
51,Dinesh Thapa,Fairmont,Thomas More University,No,Condensed Matter Physics and Material Science,Ultralow work function modulation via interfac...,The chemistry of the gold metal organic framew...,Contributed,2025-07-19,9:45,2025-07-19,10:00,ANPA2025-N00052,Sumon Hati; Rajesh Sardar; Svetlana Kilina,2025-07-19 09:45:00,2025-07-19 19:30:00,Dinesh Thapa,"In-Person Presentation, Fairmont"
15,Bishnu Sedai,Fairmont,Fairmont State University,No,"Atomic, Molecular, Optical and Plasma Physics",Spectral Shift Functions for One-Dimensional S...,"In quantum mechanics and operator theory, the ...",Contributed,2025-07-19,10:00,2025-07-19,10:15,ANPA2025-N00016,NaN,2025-07-19 10:00:00,2025-07-19 19:45:00,Bishnu Sedai,"In-Person Presentation, Fairmont"
21,Dipendra Khatri,Fairmont,University of Central Florida,No,"Atomic, Molecular, Optical and Plasma Physics",Measurement of Real-Time Evolution of Vibratio...,We present field-resolved measurements of vibr...,Contributed,2025-07-19,10:15,2025-07-19,10:30,ANPA2025-N00022,"Christopher Lantigua, Tran-Chau Truong, Chelse...",2025-07-19 10:15:00,2025-07-19 20:00:00,Dipendra Khatri,"In-Person Presentation, Fairmont"


In [49]:
create_html(df_data,html_name='html_code_2025_data.html')
create_html(df_PER,html_name='html_code_2025_PER.html')
create_html(df_Condensed,html_name='html_code_2025_Condensed.html')
#create_html(df_Bio,html_name='html_code_2024_Bio.html')
create_html(df_Plasma,html_name='html_code_2025_Plasma.html')
create_html(df_HEP,html_name='html_code_2025_HEP.html')
create_html(df_Applied,html_name='html_code_2025_Applied.html')
create_html(df_Astro,html_name='html_code_2025_Astro.html')

# Create HTML based on Location
create_html(df_CDP,html_name='html_code_2025_CDP.html')
create_html(df_FSU,html_name='html_code_2025_Fairmont.html')
create_html(df_Plenary,html_name='html_code_2025_Plenary.html')


## Create Calendar

In [50]:
df.dtypes

presenter                  object
method                     object
affiliation                object
manuscript                 object
division                   object
title                      object
abstract                   object
Type                       object
Start Date                 object
Start Time                 object
End Date                   object
End Time                   object
abstract_id                object
coauthors                  object
Date/Time          datetime64[ns]
Nepal Date/Time    datetime64[ns]
Author_I                   object
CDP/FYT/Virtual            object
dtype: object

In [51]:
df['Start_DateTime']=df['Start Date'].astype(str)+' '+df['Start Time'].astype(str)
df['Start_DateTime'] = pd.to_datetime(df['Start_DateTime'],format=('%Y-%m-%d  %H:%M'))
df['Start_DateTime'] = df['Start_DateTime'] + timedelta(hours=4)#.dt.tz_localize('UTC')#.dt.tz_convert('US/Eastern')

df['End_DateTime']=  df['End Date'].astype(str)+' '+df['End Time'].astype(str)
df['End_DateTime'] = pd.to_datetime(df['End_DateTime'],format=('%Y-%m-%d  %H:%M'))
df['End_DateTime'] = df['End_DateTime'] + timedelta(hours=4)#.dt.tz_localize('UTC')#.dt.tz_convert('US/Eastern')#

In [52]:
def prep_list(df):
    Names_L= df['Author_I'].values
    title_L = df['title'].values
    Date_L=df['Start_DateTime'].values
    EndDate_L=df['End_DateTime'].values
    return Names_L,title_L,Date_L,EndDate_L

In [53]:
df[['Start_DateTime','End_DateTime','abstract_id',"title"]].sort_values(by="Start_DateTime")

,Start_DateTime,End_DateTime,abstract_id,title
27,2025-07-17 21:00:00,2025-07-17 21:15:00,ANPA2025-N00028,Collective Chemo-sensing by Neuron Spheroids
28,2025-07-17 21:15:00,2025-07-17 21:30:00,ANPA2025-N00029,Impact of Lipid and Cholesterol Peroxidation o...
29,2025-07-17 21:30:00,2025-07-17 21:45:00,ANPA2025-N00030,Q147E Deamidation Modulates the Membrane Bindi...
30,2025-07-17 21:45:00,2025-07-17 22:00:00,ANPA2025-N00031,Single-atom catalysts supported on transition ...
37,2025-07-17 22:00:00,2025-07-17 22:15:00,ANPA2025-N00038,Monitoring of Blood Serum Spectral Biomarkers ...
...,...,...,...,...
5,2025-07-19 18:00:00,2025-07-19 18:15:00,ANPA2025-N0006,Unveiling the Nature of 3-Second ULF Waves in ...
76,2025-07-19 18:15:00,2025-07-19 18:30:00,ANPA2025-N00077,Quarkonia as the Probe of QGP formation in Hig...
84,2025-07-19 18:30:00,2025-07-19 18:45:00,ANPA2025-N00085,Transforming Online Physics Lab Courses into H...
87,2025-07-19 18:45:00,2025-07-19 19:00:00,ANPA2025-N00088,Fostering Inclusiveness in Physics for Undergr...


In [54]:
df['diff'] = (df['End_DateTime'] - df['Start_DateTime']).dt.days
df['diff'].value_counts()

0    92
Name: diff, dtype: int64

In [55]:
df.loc[df['diff']==-1]

,presenter,method,affiliation,manuscript,division,title,abstract,Type,Start Date,Start Time,...,End Time,abstract_id,coauthors,Date/Time,Nepal Date/Time,Author_I,CDP/FYT/Virtual,Start_DateTime,End_DateTime,diff


In [56]:
c=Calendar()
Names_L,title_L,Date_L,EndDate_L=prep_list(df)
str(Date_L[0])
for i in range(len(Names_L)):
    e=Event()
    e.name=Names_L[i]
    e.begin=str(Date_L[i])
    e.end=str(EndDate_L[i])
    e.description=title_L[i]
    c.events.add(e)
c.events

{<Event 'Rabina Thapa Magar' begin:2025-07-17T21:00:00+00:00 end:2025-07-17T21:15:00+00:00>,
 <Event 'Nawal K. Khadka' begin:2025-07-17T21:15:00+00:00 end:2025-07-17T21:30:00+00:00>,
 <Event 'Preston Hazen' begin:2025-07-17T21:30:00+00:00 end:2025-07-17T21:45:00+00:00>,
 <Event 'Bipin Lamichhane' begin:2025-07-17T21:45:00+00:00 end:2025-07-17T22:00:00+00:00>,
 <Event 'Sachini Herath Ekanayake' begin:2025-07-17T22:00:00+00:00 end:2025-07-17T22:15:00+00:00>,
 <Event 'Farzam Javadpour (Invited)' begin:2025-07-17T22:15:00+00:00 end:2025-07-17T22:45:00+00:00>,
 <Event 'Chandra P Joshi' begin:2025-07-17T23:15:00+00:00 end:2025-07-17T23:30:00+00:00>,
 <Event 'Uma Poudyal' begin:2025-07-17T23:30:00+00:00 end:2025-07-17T23:45:00+00:00>,
 <Event 'Sujata Bhandari' begin:2025-07-17T23:45:00+00:00 end:2025-07-18T00:00:00+00:00>,
 <Event 'Umesh Silwal' begin:2025-07-18T00:00:00+00:00 end:2025-07-18T00:15:00+00:00>,
 <Event 'Rajan Ghimire' begin:2025-07-18T00:15:00+00:00 end:2025-07-18T00:30:00+00:00

In [57]:
with open('ANPA Conference Calendar 2025.ics', 'w', encoding="utf-8") as f:
    f.write(str(c))

C:\Users\plasm\AppData\Roaming\Python\Python39\site-packages\ics\component.py:85: FutureWarning: Behaviour of str(Component) will change in version 0.9 to only return a short description, NOT the ics representation. Use the explicit Component.serialize() to get the ics representation.
  warnings.warn(


In [58]:
## Create time sorted file
df.columns

Index(['presenter', 'method', 'affiliation', 'manuscript', 'division', 'title',
       'abstract', 'Type', 'Start Date', 'Start Time', 'End Date', 'End Time',
       'abstract_id', 'coauthors', 'Date/Time', 'Nepal Date/Time', 'Author_I',
       'CDP/FYT/Virtual', 'Start_DateTime', 'End_DateTime', 'diff'],
      dtype='object')

In [59]:
df_single = df.sort_values(by = "Start_DateTime")[["Start_DateTime","Nepal Date/Time", "presenter","title"]]
df_single

,Start_DateTime,Nepal Date/Time,presenter,title
27,2025-07-17 21:00:00,2025-07-18 02:45:00,Rabina Thapa Magar,Collective Chemo-sensing by Neuron Spheroids
28,2025-07-17 21:15:00,2025-07-18 03:00:00,Nawal K. Khadka,Impact of Lipid and Cholesterol Peroxidation o...
29,2025-07-17 21:30:00,2025-07-18 03:15:00,Preston Hazen,Q147E Deamidation Modulates the Membrane Bindi...
30,2025-07-17 21:45:00,2025-07-18 03:30:00,Bipin Lamichhane,Single-atom catalysts supported on transition ...
37,2025-07-17 22:00:00,2025-07-18 03:45:00,Sachini Herath Ekanayake,Monitoring of Blood Serum Spectral Biomarkers ...
...,...,...,...,...
5,2025-07-19 18:00:00,2025-07-19 23:45:00,Poshan Belbase,Unveiling the Nature of 3-Second ULF Waves in ...
76,2025-07-19 18:15:00,2025-07-20 00:00:00,Sabin Thapa,Quarkonia as the Probe of QGP formation in Hig...
84,2025-07-19 18:30:00,2025-07-20 00:15:00,Umesh Silwal,Transforming Online Physics Lab Courses into H...
87,2025-07-19 18:45:00,2025-07-20 00:30:00,Shruti Shrestha,Fostering Inclusiveness in Physics for Undergr...


In [60]:
df_single.to_csv("date_sorted_schedule.csv",index=False)